<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021)

In [1]:
import requests
import pandas as pd
import numpy as np
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## Day 1: Sonar Sweep

In [2]:
r = requests.get(path+'AoC_01.txt')
# Ignore empty string on the end
data = r.text.split('\n')[:-1]

**Part One**: Count the number of times a depth measurement increases from the previous measurement.

In [3]:
df = pd.DataFrame([int(d) for d in data], columns=['depth'])
df['diff'] = df.depth.diff()
df

,depth,diff
0,141,NaN
1,140,-1.0
2,160,20.0
3,161,1.0
4,162,1.0
...,...,...
1995,9081,1.0
1996,9084,3.0
1997,9089,5.0
1998,9102,13.0


In [4]:
df.query('diff > 0').shape

(1692, 2)

**Part Two**: Consider sums of a three-measurement sliding window. Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum.

In [5]:
df.depth.astype('float', copy=False)

0        141.0
1        140.0
2        160.0
3        161.0
4        162.0
         ...  
1995    9081.0
1996    9084.0
1997    9089.0
1998    9102.0
1999    9105.0
Name: depth, Length: 2000, dtype: float64

In [6]:
df['window'] = df.depth.rolling(3).sum()
df['window_diff'] = df.window.diff()
df.query('window_diff > 0').shape

(1724, 4)